In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019
import time

startTime = time.time()

In [ ]:
%%time
import static_individual_estimates
static_individual_estimates.estimate_paces(ve_or_ju)

In [ ]:
%%time
static_individual_estimates.combine_estimates_with_running_order(ve_or_ju)

In [ ]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)



In [ ]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [ ]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



In [ ]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

In [ ]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

In [ ]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

In [ ]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [ ]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

In [ ]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

In [ ]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [ ]:
estimates.head()

In [ ]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

In [ ]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

In [ ]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]
local_start_ts = pd.Timestamp(start_timestamp).tz_localize(None)

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= local_start_ts)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

In [ ]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

for_print[for_print['team'].str.contains("Stadin")]
#for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

In [ ]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Downt")]

In [ ]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [ ]:
for_web.shape

In [ ]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean", "count"]).round(2)


In [ ]:
for_web["interval_length"] = for_web.fin_time_end95 - for_web.fin_time_start95
for_web["interval_length"].median()
#for_web[['leg', 'interval_length', "num_runs"]].info()

In [ ]:
for_web[['leg', 'interval_length', "num_runs"]].groupby('leg').agg(["mean", "count"]).round(2)

In [ ]:
endTime = time.time()
shared.log_df(f"{ve_or_ju} runtime {round(((endTime - startTime)/ 60), 2)} mins")